# Step 3 Calculating interaction of randomly generated particles in 3D space

In [1]:
import numpy as np
from scipy.special import lpmn, factorial
from multipole import Vlm
from multipole import operation as op

In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('ggplot')
def plot_3d(x):
    """plot particles in 3 dimentional"""
    y = np.transpose(x)
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(y[0], y[1], y[2])
    ax = ax.view_init(30)
    plt.show()
    return 

In [48]:
#case 3.1 construction: random particles
num_particles = 100
x_i = np.ndarray(shape=(3, num_particles))
x_i[0] = 20 * np.random.rand(num_particles) - 10
x_i[1] = 20 * np.random.rand(num_particles) - 10
x_i[2] = 20 * np.random.rand(num_particles) - 10
x_i = np.transpose(x_i)
q = np.ones(num_particles)

In [98]:
#case 3.2 construction: nearly uniform distributed particles:
num_particles_in_a_box = 1
num_particels_1D = 3
num_particles = num_particles_in_a_box * num_particels_1D **3
X = np.zeros(shape=(num_particles,3))

for i in range(0,num_particels_1D):
    for j in range(0,num_particels_1D):
        for k in range(0,num_particels_1D):
            X[i*num_particels_1D*num_particels_1D+j*num_particels_1D+k] = [i,j,k] 

q = np.ones(num_particles)

In [ ]:
plot_3d(x0_i)

In [99]:
[x0_i, scale_factor] = op.cartesian_scaling_to_unit_range(X)

import timeit

start = timeit.default_timer()

[x0_i, scale_factor] = op.cartesian_scaling_to_unit_range(x_i)

stop = timeit.default_timer()

print(stop - start) 

In [100]:
# analytical answer
pair_potential = np.zeros(shape=(num_particles,num_particles)) 
for i in range(0, num_particles):
    for j in range(i+1, num_particles):
        pair_potential[i][j] = 1 / (op.distance_cal(x0_i[i], x0_i[j]) * scale_factor[1])
        
pair_potential

array([[ 0.        ,  1.000001  ,  0.5000005 ,  1.000001  ,  0.70710749,
         0.44721404,  0.5000005 ,  0.44721404,  0.35355374,  1.000001  ,
         0.70710749,  0.44721404,  0.70710749,  0.57735085,  0.4082487 ,
         0.44721404,  0.4082487 ,  0.33333367,  0.5000005 ,  0.44721404,
         0.35355374,  0.44721404,  0.4082487 ,  0.33333367,  0.35355374,
         0.33333367,  0.28867542],
       [ 0.        ,  0.        ,  1.000001  ,  0.70710749,  1.000001  ,
         0.70710749,  0.44721404,  0.5000005 ,  0.44721404,  0.70710749,
         1.000001  ,  0.70710749,  0.57735085,  0.70710749,  0.57735085,
         0.4082487 ,  0.44721404,  0.4082487 ,  0.44721404,  0.5000005 ,
         0.44721404,  0.4082487 ,  0.44721404,  0.4082487 ,  0.33333367,
         0.35355374,  0.33333367],
       [ 0.        ,  0.        ,  0.        ,  0.44721404,  0.70710749,
         1.000001  ,  0.35355374,  0.44721404,  0.5000005 ,  0.44721404,
         0.70710749,  1.000001  ,  0.4082487 ,  0.5773

In [101]:
J_analytic = np.zeros(num_particles)
for i in range(0, num_particles):
    for j in range(0, num_particles):
        if j<i:
            J_analytic[i] += pair_potential[j][i]
        if j>i:
            J_analytic[i] += pair_potential[i][j]

J_analytic

array([ 13.45604582,  15.02674426,  13.45604582,  15.02674426,
        16.88811976,  15.02674426,  13.45604582,  15.02674426,
        13.45604582,  15.02674426,  16.88811976,  15.02674426,
        16.88811976,  19.10410263,  16.88811976,  15.02674426,
        16.88811976,  15.02674426,  13.45604582,  15.02674426,
        13.45604582,  15.02674426,  16.88811976,  15.02674426,
        13.45604582,  15.02674426,  13.45604582])

In [102]:
total_energy = 0.5 * sum(J_analytic)
total_energy

204.20105944767533

In [103]:
from multipole import fmm_level as fl
from multipole import fmm_q_source as fq

In [104]:
# build list of q_source
q_source = np.ndarray(shape=(len(x0_i)), dtype=fq) 
for i in range(0, len(x0_i)):
    q_source[i] = fq(x0_i[i], q[i])
    

In [130]:
# run the calculation:
#1 construction of boxes at each level with Olm
btm_level = 3
p = 10
ws_index = 4
f_btm_level = fl(btm_level, q_source, 5, ws_index)

f_top_level = f_btm_level
while (f_top_level.level != 1):
    print(f_top_level.level)
    f_top_level = f_top_level.lower_level_construction()
    
#2 calculation the interaction and translation the potential
print('----------')
f_level_i = f_top_level
while (f_level_i.level != f_btm_level.level):
    f_level_i.Mlm_translation_to_higher_level()
    f_level_i = f_level_i.higher_level
    print(f_level_i.level)
    f_level_i.box_interactions()
    
f_level_i.level

3
2
----------
2
3


3

In [131]:
#3 calculation of J far field
J_far_field = np.zeros(num_particles)
for i in range(0, num_particles):
    if not f_btm_level.box_list[q_source[i].box_id].Mlm:
        J_far_field[i] = 0.
    else:
        J_far_field[i] = f_btm_level.box_list[q_source[i].box_id].Mlm.product(q_source[i].Olm).sum().real 

J_far_field /= scale_factor[1]

In [132]:
J_far_field

array([ 8.8184464 ,  7.24548657,  8.68962549,  7.00974925,  4.17229221,
        6.94535621,  8.73696041,  7.18809017,  8.61024151,  7.40145023,
        4.76418802,  7.32211245,  4.40013171,  0.        ,  4.40013362,
        7.33644176,  4.76418328,  7.25710409,  8.59579249,  7.17352594,
        8.47273741,  6.82971383,  4.17229628,  6.76532121,  8.51314684,
        7.11612927,  8.39219374])

In [133]:
#4 calculation of J near filed
J_near_field = np.zeros(num_particles)
for i in range(0,num_particles):
    J_near_field[i] = 0.
    for j in f_btm_level.box_list[q_source[i].box_id].q_source_id_set:
        if j == i:
            continue
        J_near_field[i] += 1  / (op.distance_cal(q_source[i].x, q_source[j].x) * scale_factor[1])
    for NN_box_id in f_btm_level.box_list[q_source[i].box_id].NN_box_id_set:
        for j in f_btm_level.box_list[NN_box_id].q_source_id_set:
            J_near_field[i] += 1  / (op.distance_cal(q_source[i].x, q_source[j].x) * scale_factor[1])

In [134]:
J_near_field

array([  5.69867631,   8.69024313,   5.69867631,   8.69024313,
        12.96626829,   8.69024313,   5.69867631,   8.69024313,
         5.69867631,   8.69024313,  12.96626829,   8.69024313,
        12.96626829,  19.10410263,  12.96626829,   8.69024313,
        12.96626829,   8.69024313,   5.69867631,   8.69024313,
         5.69867631,   8.69024313,  12.96626829,   8.69024313,
         5.69867631,   8.69024313,   5.69867631])

In [135]:
J_total = J_far_field + J_near_field
J_total

array([ 14.51712271,  15.93572971,  14.38830181,  15.69999238,
        17.1385605 ,  15.63559935,  14.43563672,  15.8783333 ,
        14.30891782,  16.09169336,  17.73045631,  16.01235558,
        17.3664    ,  19.10410263,  17.36640191,  16.0266849 ,
        17.73045157,  15.94734722,  14.2944688 ,  15.86376908,
        14.17141373,  15.51995696,  17.13856457,  15.45556434,
        14.21182315,  15.8063724 ,  14.09087005])

In [136]:
total_energy = 0.5 * sum(J_total)
total_energy

213.93344544061151

In [129]:
J_error = np.abs(J_total-J_analytic) / J_analytic
J_error

array([  7.88550294e-02,   6.04911770e-02,   6.92815704e-02,
         4.48033257e-02,   1.48294037e-02,   4.05180972e-02,
         7.27993139e-02,   5.66715598e-02,   6.33820673e-02,
         7.08702485e-02,   4.98774621e-02,   6.55904768e-02,
         2.83205148e-02,   1.85966007e-16,   2.83206278e-02,
         6.65440642e-02,   4.98771816e-02,   6.12642995e-02,
         6.23082731e-02,   5.57023397e-02,   5.31633079e-02,
         3.28223259e-02,   1.48296446e-02,   2.85371251e-02,
         5.61663763e-02,   5.18827048e-02,   4.71776212e-02])

In [ ]:
for i in range(0,len(f_btm_level.box_list)):
    if f_btm_level.box_list[i]:
        print(i)

In [ ]:
index1 = 27
index2 = 465


Y = np.ndarray(shape=(4,3)) #Cartesian coordiantes of box centers
Y[0] = f_btm_level.box_list[index1].x
Y[1] = f_btm_level.box_list[index2].x
Y[2] = f_btm_level.lower_level.box_list[f_btm_level.box_id_to_lower_level(index1)].x
Y[3] = f_btm_level.lower_level.box_list[f_btm_level.box_id_to_lower_level(index2)].x

y_i = np.ndarray(shape=(2,3))
y_i[0] = q_source[0].x
y_i[1] = q_source[1].x


In [ ]:
Y

In [ ]:
analytic_potential = 0 
for i in range(0, len(y_i)):
    for j in range(i+1, len(y_i)):
        analytic_potential += 1 / op.distance_cal(y_i[i], y_i[j])

analytic_potential / scale_factor[1]

In [ ]:
# test on conversion operation
p1 = 5

r1_1 = op.cartesian_to_spherical(y_i[0] - Y[0])
Olm_q1_x1_k = op.O_expansion(p1, r1_1)

Olm_q1_x1_k.Vp

In [ ]:
q_source[0].Olm.Vp

In [ ]:
Y31 = Y[0] - Y[2]
Olm_q1_x3_k = op.O_to_O(Olm_q1_x1_k, Y31)

Olm_q1_x3_k.Vp

In [ ]:
f_btm_level.lower_level.box_list[f_btm_level.box_id_to_lower_level(index1)].Olm.Vp

In [ ]:
Y34 = Y[3] - Y[2]
Mlm_q1_x4_k = op.O_to_M(Olm_q1_x3_k, Y34)
Mlm_q1_x4_k.Vp

In [ ]:
f_btm_level.lower_level.box_list[f_btm_level.box_id_to_lower_level(index2)].Mlm.Vp

In [ ]:
Y42 = Y[1] - Y[3]
Mlm_q1_x2_k = op.M_to_M(Mlm_q1_x4_k, Y42)

Mlm_q1_x2_k.Vp

In [ ]:
f_btm_level.box_list[index2].Mlm.Vp